In [ ]:
%pip install duckdb numpy pandas

In [1]:
import pandas as pd

import duckdb

conn = duckdb.connect("main.db")

In [2]:
file_path = "/workspaces/Retail-Growth-Analysis-Project/Dataset"

In [6]:
conn.execute(f"""CREATE OR REPLACE TABLE transactions 
    AS 
    SELECT * FROM read_csv("{file_path}/ebay_transactions.csv")
    """)

conn.execute("SELECT * FROM transactions LIMIT 3").fetch_df()

,id,transaction_id,date,customer_id,product_id,quantity,total_revenue
0,1,TXN-100000,2023-12-13,CUST-7180,PROD-1003,3,1576.92
1,2,TXN-100001,2023-10-12,CUST-8266,PROD-1055,4,626.52
2,3,TXN-100002,2023-12-01,CUST-7519,PROD-1066,4,1736.84


In [7]:
conn.execute(f"""CREATE OR REPLACE TABLE customers 
    AS 
    SELECT * FROM read_csv("{file_path}/ebay_customers.csv")
    """)

conn.execute("SELECT * FROM customers LIMIT 3").fetch_df()

,id,customer_id,customer_name,region,segment
0,1,CUST-5000,Melinda Cameron,West,Home Office
1,2,CUST-5001,Crystal Johnson,West,Corporate
2,3,CUST-5002,Derek Clark,South,Consumer


In [8]:
conn.execute(f"""CREATE OR REPLACE TABLE products 
    AS 
    SELECT * FROM read_csv("{file_path}/ebay_products.csv")
    """)

conn.execute("SELECT * FROM products LIMIT 3").fetch_df()

,id,product_id,product_name,category,unit_price,cost_price
0,1,PROD-1000,Books Purpose,Books,306.57,186.91
1,2,PROD-1001,Electronics Brother,Electronics,219.32,116.46
2,3,PROD-1002,Sports Ago,Sports,630.04,362.24


### What was the total gross revenue generated in the month of December?

In [3]:
conn.execute("""
        SELECT 
            ROUND(SUM(total_revenue),0) AS total_revenue_dec_23         
        FROM transactions
        WHERE date BETWEEN '2023-12-01' AND '2023-12-31'
""").fetch_df()

,total_revenue_dec_23
0,6522413.0


In [4]:
conn.execute("""
        SELECT 
            EXTRACT('month' FROM date) AS mth,
            ROUND(SUM(total_revenue),0) AS total_revenue       
        FROM transactions
        GROUP BY mth
""").fetch_df()

,mth,total_revenue
0,10,6442830.0
1,11,6182842.0
2,12,6522413.0


### Which Product Category sold the highest total number of units (Quantity) across the entire quarter?

In [5]:
conn.execute("""
        SELECT 
            category,
            SUM(quantity) AS total_units_solds      
        FROM transactions
        JOIN products
             ON transactions.product_id = products.product_id
        GROUP BY category
""").fetch_df()

,category,total_units_solds
0,Clothing,11672.0
1,Home & Kitchen,9834.0
2,Sports,9044.0
3,Books,7270.0
4,Electronics,6966.0


### Which Product Category yielded the highest Net Profit?

(Calculation: Total Revenue - (Cost Price * Quantity)).

In [7]:
conn.execute("""
        SELECT 
            category,
            SUM(total_revenue) AS total_revenue,
            SUM(cost_price * quantity) AS COGS,
            SUM(total_revenue) - SUM(cost_price * quantity) AS net_profit  
        FROM transactions
        JOIN products
             ON transactions.product_id = products.product_id
        GROUP BY category
        ORDER BY net_profit DESC
""").fetch_df()

,category,total_revenue,COGS,net_profit
0,Clothing,4650984.77,2575943.03,2075041.74
1,Sports,4050820.11,2203615.12,1847204.99
2,Home & Kitchen,4013454.91,2219284.32,1794170.59
3,Books,3168017.06,1796712.72,1371304.34
4,Electronics,3264808.47,1952093.24,1312715.23


### Which Region had the highest Average Order Value (AOV)?

In [8]:
conn.execute("""
        SELECT 
            region,
            AVG(total_revenue) AS aov
        FROM transactions
        JOIN customers
             ON transactions.customer_id = customers.customer_id
        GROUP BY region
        ORDER BY aov DESC
""").fetch_df()

,region,aov
0,West,1291.385807
1,East,1284.290895
2,North,1265.495041
3,South,1263.891298


### How many Unique Customers made at least one purchase in both October and December?

In [13]:
conn.execute("""
        WITH unique_customers_oct_dec AS (
        SELECT 
            DISTINCT customer_id
        FROM transactions
        WHERE date BETWEEN '2023-10-01' AND '2023-11-01'
             
        UNION
        
        SELECT 
            DISTINCT customer_id
        FROM transactions
        WHERE date BETWEEN '2023-12-01' AND '2024-01-01'
             )

        SELECT
            COUNT(*)
        FROM unique_customers_oct_dec
""").fetch_df()

,count_star()
0,3339


###  List the names of the Top 3 Products based on total revenue generated.


In [16]:
conn.execute("""
        SELECT 
            product_name,
            SUM(total_revenue) AS total_revenue
        FROM transactions
        JOIN products
             ON transactions.product_id = products.product_id
        GROUP BY product_name
        ORDER BY product_name DESC
        LIMIT 3
        
""").fetch_df()

,product_name,total_revenue
0,Sports Wish,59698.34
1,Sports True,87948.63
2,Sports Training,68496.24
